In [1]:
import h5py
import numpy as np
from astropy.io import fits
import os
from tqdm import tqdm
import torch
import argparse



In [14]:
apogee_dir="./test2/apogee"
galah_dir="./test2/galah"
combined_file="./test/combined.fits"
hdf5_path="./test2/data.hdf5"
latent_size=20
catalog_path='/arc/projects/k-pop/catalogues/galah-dr3v2.fits'

In [2]:

def load_galah_catalog(catalog_path):
    with fits.open(catalog_path) as hdul:
        data = hdul[1].data
    return {row['sobject_id']: row for row in data}
  

In [17]:
galah_catalog =  load_galah_catalog(catalog_path)


In [19]:
first_five_entries = {k: galah_catalog[k] for k in list(galah_catalog)[:5]}
for id, entry in first_five_entries.items():
    print(f"{id}: {entry}")

131116000501002: ('03325271-6840304', 131116000501002, 4667368899326729856, 4667368899326729856, 'galah_main', 138, 0, '10k_2', 'lbol', 0, 4553.3545, 137.06088556228917, 4138.0, 0.048, 'sfd', 4.7149534, 0.20121636742743593, -0.33501863, 0.12021913955493135, 0, -0.109347336, 0.94853675, 9.345594, 2.5080910007092827, 0.5873856, 0.35136763119226394, 0.07919326531297675, 405.0, 0, 0.73446965, 0.80794984, -0.18375580310821538, 0.12854836388714738, 1, 1, nan, nan, -9223372036854775808, 32, 1.2146472740173344, 0.2138376266813869, 1, 1, 0.14526838302612344, 0.11489051187565578, 1, 0, -0.08849821090698207, 0.19155067846881743, 1, 0, 0.46733125686645494, 0.11861814440786478, 1, 0, nan, nan, 1, 32, 0.4403888511657712, 0.1647182437596357, 1, 0, 0.40222770690917997, 0.12761818533447994, 1, 0, 0.427332248687744, 0.10620845786539221, 1, 0, nan, nan, 0, 0, 0.6909773123929537, 0.20731252088935498, 18, 0, nan, nan, -9223372036854775808, 32, 0.8553736305236814, 0.3077087728097469, 2, 0, 0.242551441192627

In [33]:
specific_id = "131118002901377"
if specific_id in galah_catalog:
    print(f"Data for sobject_id {specific_id}:")
    print(galah_catalog[specific_id])
else:
    print(f"sobject_id {specific_id} not found in the catalog.")

sobject_id 131118002901377 not found in the catalog.


In [3]:
  
def ensure_native_byteorder(array):
    if array.dtype.byteorder not in ('=', '|'):
        return array.byteswap().newbyteorder()
    return array


In [4]:

def calculate_wavelength_apogee(header, flux):
    crval = header['CRVAL1']
    cdelt = header['CDELT1']
    crpix = header['CRPIX1']
    n_pixels = len(flux)
    index = np.arange(n_pixels)
    return 10 ** (crval + (index - (crpix - 1)) * cdelt)

def calculate_wavelength_galah(header, flux):
    w_start = header['CRVAL1']  # Starting wavelength
    w_delta = header['CDELT1']  # Wavelength increment per pixel
    n_pixels = len(flux)
    wavelength = w_start + w_delta * np.arange(n_pixels) 
    return wavelength


In [5]:

def create_mask_apogee(flux, sigma):
    mask = np.where((flux == 0) | (sigma > 0.02), 0, 1)
    return mask

def create_mask_galah(flux, sigma):
    mask = np.where((flux == 0), 0, 1)
    return mask



In [6]:

def extract_apogee_id(filename):
    prefix = 'aspcapStar-dr17-'
    suffix = '.fits'
    if filename.startswith(prefix) and filename.endswith(suffix):
        return filename[len(prefix):-len(suffix)]
    return None



In [7]:

def extract_apogee_metadata(file_path):
    with fits.open(file_path) as hdul:
        header = hdul[4].header  
        return {
            'temp': header.get('TEFF', None),
            'temp_err': header.get('TEFF_ERR', None),
            'logg': header.get('LOGG', None),
            'logg_err': header.get('LOGG_ERR', None),
            'fe_h': header.get('FE_H', None),
            'fe_h_err': header.get('FE_H_ERR', None),
            'rv': header.get('VHELIO_AVG', None),
            'rv_err': header.get('VERR', None),
            'ra': header.get('RA', None),
            'dec' : header.get('DEC', None)
        }


In [30]:

def extract_galah_metadata(galah_catalog, sobject_id):
    if sobject_id in galah_catalog:
        row = galah_catalog[sobject_id]
        return {
            'temp': row['teff'],
            'temp_err': row['e_teff'],
            'logg': row['logg'],
            'logg_err': row['e_logg'],
            'fe_h': row['fe_h'],
            'fe_h_err': row['e_fe_h'],
            'rv': row['rv_galah'],
            'rv_err': row['e_rv_galah'],
            'ra': row['ra_dr2'],
            'dec': row ['dec_dr2']
        }
    return None
 

In [31]:
galah_meta = extract_galah_metadata(galah_catalog, 131118003401123)
print(galah_meta)

{'temp': 5039.3027, 'temp_err': 97.47001250027941, 'logg': 2.9666305, 'logg_err': 0.1993773010034177, 'fe_h': -1.1405907, 'fe_h_err': 0.08656646779203103, 'rv': 317.6990051269531, 'rv_err': 0.12999999523162842, 'ra': 92.35567615458831, 'dec': -59.65332432159097}


In [9]:
   
def save_to_hdf5(flux, wavelength, sigma, flux_mask, unique_id, instrument_type, hdf5_file, index, latent_size, combined_meta):
    latent_code = torch.normal(0, 1, size=(latent_size,), dtype=torch.float32).numpy()
    grp = hdf5_file.create_group(unique_id)
    grp.create_dataset('flux', data=flux, compression="gzip", compression_opts=9)
    grp.create_dataset('wavelength', data=wavelength, compression="gzip", compression_opts=9)
    grp.create_dataset('sigma', data=sigma, compression="gzip", compression_opts=9)
    grp.create_dataset('flux_mask', data=flux_mask)
    grp.create_dataset('latent_code', data=latent_code)
    grp.create_dataset('instrument_type', data=np.string_(instrument_type))
    
    # Handle metadata
    for key, value in combined_meta.items():
        try:
            if value is None:
                # Assuming all None values can be stored as np.nan in the dataset
                grp.create_dataset(key, data=np.array([np.nan], dtype=np.float32))
            elif isinstance(value, list):
                # Convert None to np.nan for float compatibility in HDF5
                clean_value = [np.nan if v is None else v for v in value]
                grp.create_dataset(key, data=np.array(clean_value, dtype=np.float32))
            elif isinstance(value, (int, float)):
                # Single numeric values
                grp.create_dataset(key, data=np.float32(value))
            elif isinstance(value, str):
                # Single string values
                grp.create_dataset(key, data=np.string_(value))
            elif isinstance(value, np.ndarray):
                # Directly handle numpy arrays (for instruments)
                grp.create_dataset(key, data=value, dtype=value.dtype)
            else:
                print(f"Unsupported data type for key {key}: {type(value)}")
        except Exception as e:
            print(f"Error while saving {key}: {str(e)}")




In [10]:

def load_apogee_spectrum(file_path):
    with fits.open(file_path) as hdul:
        flux = hdul[1].data.astype(np.float32)
        header = hdul[1].header
        wavelength = calculate_wavelength_apogee(header, flux).astype(np.float32)
        sigma = hdul[2].data.astype(np.float32)
        instruments = np.array([0, 1, 2], dtype=np.int32)
    
    flux = ensure_native_byteorder(flux)
    sigma = ensure_native_byteorder(sigma)
    wavelength = ensure_native_byteorder(wavelength)
    flux_mask = create_mask_apogee(flux, sigma).astype(np.float32)

    return flux, wavelength, sigma, flux_mask, instruments




In [11]:

def load_galah_spectrum(galah_dir, galah_id):
    fluxes = []
    wavelengths = []
    sigmas = []
    flux_masks = []
    instruments = []
    
    
    for i in range (1,5):
        file_path = os.path.join(galah_dir, f"{galah_id}{i}.fits")
        

        if not os.path.exists(file_path):
            print(f"Warning: File not found for {file_path}")
            continue

        instruments = np.append(instruments , i+2)
        with fits.open(file_path) as hdul:
            flux = hdul[0].data.astype(np.float32)
            header = hdul[0].header
            wavelength = calculate_wavelength_galah(header, flux).astype(np.float32)
            sigma = hdul[1].data.astype(np.float32)
        
        flux = ensure_native_byteorder(flux)
        sigma = ensure_native_byteorder(sigma)
        wavelength = ensure_native_byteorder(wavelength)
        flux_mask = create_mask_galah(flux, sigma).astype(np.float32)
        
        fluxes.append(flux)
        wavelengths.append(wavelength)
        sigmas.append(sigma)
        flux_masks.append(flux_mask)

    if not fluxes:
        raise ValueError(f"No valid GALAH files found for {galah_id}")

    return (np.concatenate(fluxes), 
            np.concatenate(wavelengths), 
            np.concatenate(sigmas), 
            np.concatenate(flux_masks), 
            instruments)




In [ ]:
def extract_apogee_id(filename):
    prefix = 'aspcapStar-dr17-'
    suffix = '.fits'
    if filename.startswith(prefix) and filename.endswith(suffix):
        return filename[len(prefix):-len(suffix)]
    return None

In [ ]:

def process_spectra(apogee_dir, galah_dir, apogee_ids, galah_ids, hdf5_file, latent_size, catalog_path):
    index = 0
    processed_apogee = set()
    processed_galah = set()
    galah_catalog = load_galah_catalog(catalog_path)
    for i, (apogee_id, galah_id) in enumerate(tqdm(zip(apogee_ids, galah_ids), total=len(apogee_ids), desc="Converting spectra to HDF5")):
        apogee_file = os.path.join(apogee_dir, f"aspcapStar-dr17-{apogee_id}.fits")
        apogee_exists = os.path.exists(apogee_file)

        # try:
        #     if apogee_exists:
        #         apogee_data = load_apogee_spectrum(apogee_file)
        #         apogee_meta = extract_apogee_metadata(apogee_file)
        #     else:
        #         apogee_data = None
        #         apogee_meta = None

        #     print("looking for galah id: " , galah_id)
        #     galah_data = load_galah_spectrum(galah_dir, galah_id)
        #     galah_meta = extract_galah_metadata(galah_catalog, galah_id)

        #     if galah_meta is None:
        #         print(f"Warning: No GALAH metadata found for sobject_id {galah_id}")
        #         continue

        #     if apogee_exists:
        #         # Combine spectral data
        #         flux = np.concatenate((apogee_data[0], galah_data[0]))
        #         wavelength = np.concatenate((apogee_data[1], galah_data[1]))
        #         sigma = np.concatenate((apogee_data[2], galah_data[2]))
        #         flux_mask = np.concatenate((apogee_data[3], galah_data[3]))
        #         instruments = np.concatenate((apogee_data[4], galah_data[4]))  
        #         instrument_type = "combined"
        #     else:
        #         flux, wavelength, sigma, flux_mask, instruments = galah_data
        #         instrument_type = "galah"

        #     combined_meta = {
        #         'temp': [apogee_meta['temp'] if apogee_meta else None, galah_meta['temp']],
        #         'temp_err': [apogee_meta['temp_err'] if apogee_meta else None, galah_meta['temp_err']],
        #         'logg': [apogee_meta['logg'] if apogee_meta else None, galah_meta['logg']],
        #         'logg_err': [apogee_meta['logg_err'] if apogee_meta else None, galah_meta['logg_err']],
        #         'rv': [apogee_meta['rv'] if apogee_meta else None, galah_meta['rv']],
        #         'rv_err': [apogee_meta['rv_err'] if apogee_meta else None, galah_meta['rv_err']],
        #         'ra': [apogee_meta['ra'] if apogee_meta else None, galah_meta['ra']],
        #         'dec': [apogee_meta['dec'] if apogee_meta else None, galah_meta['dec']],
        #         'instruments': instruments,
        #         'obj_class' : "Star"
        #     }


        #     unique_id = f"{instrument_type}_{index}"
        #     index += 1

            if apogee_exists:
                processed_apogee.add(apogee_id)
            processed_galah.add(galah_id)

        except ValueError as e:
            print(f"Skipping {galah_id}: {str(e)}")
            continue

       

        # save_to_hdf5(flux, wavelength, sigma, flux_mask, unique_id, instrument_type, hdf5_file, index, latent_size, combined_meta)

        

    # Process remaining APOGEE files
    all_apogee_files = {extract_apogee_id(file) for file in os.listdir(apogee_dir) if file.endswith('.fits')}
    remaining_apogee = all_apogee_files - processed_apogee
    print('remaining_apogee = ' , remaining_apogee)
    # for apogee_id in remaining_apogee:
    #     apogee_file = os.path.join(apogee_dir, f"aspcapStar-dr17-{apogee_id}.fits")
    #     print("apogee file " , apogee_file)
    #     flux, wavelength, sigma, flux_mask, instruments = load_apogee_spectrum(apogee_file)
    #     unique_id = f"apogee_{index}"
    #     instrument_type = "apogee"
    #     index += 1

    #     apogee_meta = extract_apogee_metadata(apogee_file)
    #     combined_meta = {
            
    #         'temp': [apogee_meta['temp'], None],
    #         'temp_err': [apogee_meta['temp_err'], None],
    #         'logg': [apogee_meta['logg'], None],
    #         'logg_err': [apogee_meta['logg_err'], None],
    #         'fe_h': [apogee_meta['fe_h'], None],
    #         'fe_h_err': [apogee_meta['fe_h_err'], None],
    #         'rv': [apogee_meta['rv'], None],
    #         'rv_err': [apogee_meta['rv_err'], None],
    #         'alpha_m': apogee_meta['alpha_m'],
    #         'alpha_m_err': apogee_meta['alpha_m_err'],
    #         'ra': apogee_meta['ra'],
    #         'dec': apogee_meta['dec'],
    #         'instruments': instruments,
    #         'obj_class' : "Star"
            
    #     }


        # save_to_hdf5(flux, wavelength, sigma, flux_mask,  unique_id, instrument_type, hdf5_file, index, latent_size, combined_meta)

    # Process remaining GALAH files
    all_galah_files = {int(file[:15]) for file in os.listdir(galah_dir) if file.endswith('.fits')}
    remaining_galah = all_galah_files - processed_galah
    # for galah_id in remaining_galah:
    #     try:
    #         # print("looking for galah id remaining: " , galah_id)
    #         flux, wavelength, sigma, flux_mask, instruments = load_galah_spectrum(galah_dir, galah_id)
    #         unique_id = f"galah_{index}"
    #         instrument_type = "galah"
    #         index += 1

    #         galah_meta = extract_galah_metadata(galah_catalog, galah_id)
    #         if galah_meta is None:
    #             print(f"Warning: No GALAH metadata found for sobject_id {galah_id}")
    #             continue

    #         combined_meta = {
    #             'temp': [None, galah_meta['temp']],
    #             'temp_err': [None, galah_meta['temp_err']],
    #             'logg': [None, galah_meta['logg']],
    #             'logg_err': [None, galah_meta['logg_err']],
    #             'fe_h': [None, galah_meta['fe_h']],
    #             'fe_h_err': [None, galah_meta['fe_h_err']],
    #             'rv': [None, galah_meta['rv']],
    #             'rv_err': [None, galah_meta['rv_err']],
    #             'ra': [None, galah_meta['ra']],
    #             'dec': [None, galah_meta['dec']],
    #             'instruments': instruments,
    #             'obj_class' : "Star"
    #         }


            # save_to_hdf5(flux, wavelength, sigma, flux_mask, unique_id, instrument_type, hdf5_file, index, latent_size, combined_meta)

        except ValueError as e:
            print(f"Skipping {galah_id}: {str(e)}")
            continue



In [ ]:
process_spectra(apogee_dir, galah_dir, apogee_ids, galah_ids, hdf5_file, latent_size, catalog_path)

In [13]:

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Convert APOGEE and GALAH FITS files to combined HDF5")
    parser.add_argument('--apogee_dir', type=str, required=True, help="Directory containing APOGEE FITS files")
    parser.add_argument('--galah_dir', type=str, required=True, help="Directory containing GALAH FITS files")
    parser.add_argument('--combined_file', type=str, required=True, help="Path to combined FITS catalog")
    parser.add_argument('--hdf5_path', type=str, required=True, help="Path to output HDF5 file")
    parser.add_argument('--latent_size', type=int, default=10, help="Size of the latent code")
    parser.add_argument('--catalog_path', type=str, required=True, help="Path to Galah catalog fits file")

    args = parser.parse_args()
    with h5py.File(args.hdf5_path, 'w') as hdf5_file, fits.open(args.combined_file) as hdul:
        data = hdul[1].data
        apogee_ids = data['APOGEE_ID']
        galah_ids = data['sobject_id']

        process_spectra(args.apogee_dir, args.galah_dir, apogee_ids, galah_ids, hdf5_file, args.latent_size , args.catalog_path)

usage: ipykernel_launcher.py [-h] --apogee_dir APOGEE_DIR --galah_dir
                             GALAH_DIR --combined_file COMBINED_FILE
                             --hdf5_path HDF5_PATH [--latent_size LATENT_SIZE]
                             --catalog_path CATALOG_PATH
ipykernel_launcher.py: error: the following arguments are required: --apogee_dir, --galah_dir, --combined_file, --hdf5_path, --catalog_path


SystemExit: 2

/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
